In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout
from keras.datasets import mnist
from keras.optimizers import Adam
import numpy as np
import os
from PIL import Image

if not 'savefig' in os.listdir():
    os.mkdir('savefig')

Using TensorFlow backend.


### 🐍mnistデータ読み込む🐍
- 読み込んだ後に画像の画素値を-1～1に正規化した

In [0]:
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

batch_size = 256

half_batch = int(batch_size / 2)

### 🐍最適化関数はAdamに🐍
- なんでAdamにしたか？[ここ](https://www.renom.jp/ja/notebooks/tutorial/basic_algorithm/adam/notebook.html)参照してください。
- とても長いので集約すると、現時点では最も適切なパラメータ更新ができる最適化手法として認められているから。

In [3]:
z_dim = 100

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

optimizer =Adam(0.0002, 0.5)

Instructions for updating:
Colocations handled automatically by placer.


### 🐍Generatorを設計🐍

In [0]:
noise_shape = (z_dim,)
generator = Sequential()

generator.add(Dense(256, input_shape=noise_shape))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod(img_shape), activation='tanh'))
generator.add(Reshape(img_shape))

### 🐍Discriminatorを設計🐍

In [0]:
img_shape = (img_rows, img_cols, channels)
discriminator = Sequential()

discriminator.add(Flatten(input_shape=img_shape))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))

In [0]:
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### 🐍combinedを設計🐍
- GeneratorとDiscriminatorをつなぎ合わせたものを作成しておく
- Generatorを学習させるときは、Generator単体ではなく、このCombinedを使って学習を行う

In [0]:
discriminator.trainable = False
combined = Sequential([generator, discriminator])
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

### 🐍学習🐍
- 規定回数ごとに画像を保存している
- ちょっとあとでもう少しわかりやすくします
- 最終的には、Discriminatorの正答率が50%になるくらいまで、Generatorに成長してもらいたい

In [8]:
for epoch in range(50000):
    
    # 以下、Discriminator学習
    # Generatorによる偽データの生成
    noise = np.random.normal(0, 1, (half_batch, z_dim))
    gen_imgs = generator.predict(noise)
    
    # バッチサイズの半数を教師データからピックアップ
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]
    
    # discriminator学習
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    
    # それぞれの損失関数を平均
    d_loss_mean = np.add(d_loss_real, d_loss_fake) * 0.5
    
    # 以下、Generator学習
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    if epoch % 100 == 0:
        gen_imgs = np.squeeze(gen_imgs)
        save_img = gen_imgs[0] * 127.5 + 127.5
        Image.fromarray(save_img.astype(np.uint8)).save('savefig/{}.png'.format(str(epoch)))
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss_mean[0], 100*d_loss_mean[1], g_loss))

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.373744, acc.: 34.38%] [G loss: 0.888522]
100 [D loss: 0.409332, acc.: 82.03%] [G loss: 2.390654]
200 [D loss: 0.558954, acc.: 68.36%] [G loss: 2.200600]
300 [D loss: 0.710774, acc.: 43.36%] [G loss: 0.653242]
400 [D loss: 0.665071, acc.: 50.39%] [G loss: 0.673348]
500 [D loss: 0.648556, acc.: 59.77%] [G loss: 0.719629]
600 [D loss: 0.631534, acc.: 62.89%] [G loss: 0.749495]
700 [D loss: 0.629931, acc.: 60.55%] [G loss: 0.792484]
800 [D loss: 0.652421, acc.: 54.30%] [G loss: 0.804566]
900 [D loss: 0.645339, acc.: 61.33%] [G loss: 0.909680]
1000 [D loss: 0.533728, acc.: 86.33%] [G loss: 1.028529]
1100 [D loss: 0.570311, acc.: 75.39%] [G loss: 0.968905]
1200 [D loss: 0.583524, acc.: 74.22%] [G loss: 1.007975]
1300 [D loss: 0.544421, acc.: 79.30%] [G loss: 1.089133]
1400 [D loss: 0.626116, acc.: 64.84%] [G loss: 1.052416]
1500 [D loss: 0.529019, acc.: 79.69%] [G loss: 1.092259]
1600 [D loss: 0.566054, acc.: 72.27%] [G loss: 1.084102]
1700 [D loss: 0.590904, acc.: 72.27%] [G lo